In [0]:
%sql
USE hive_metastore.bd_becomex

In [0]:
df = spark.sql("""

WITH ncm_joined AS (
    SELECT 
    n.CO_NCM
    ,n.NO_NCM_POR
    ,ni.NO_ISIC_CLASSE
    ,ni.NO_ISIC_GRUPO
    ,ni.CO_ISIC_DIVISAO
    ,ni.NO_ISIC_DIVISAO
    ,ni.NO_ISIC_SECAO
    ,s.NO_SH6_POR
    ,s.CO_SH4
    ,s.NO_SH4_POR
    ,s.CO_SH2
    ,s.NO_SH2_POR
    ,s.CO_NCM_SECROM
    ,s.NO_SEC_POR
    ,np.NO_PPE
    ,npi.NO_PPI
    ,nfa.NO_FAT_AGREG
    ,nc.NO_CUCI_ITEM
    ,nc.CO_CUCI_SUB
    ,nc.NO_CUCI_SUB
    ,nc.CO_CUCI_GRUPO
    ,nc.NO_CUCI_SEC
    ,ncg.NO_CGCE_N3
    ,ncg.NO_CGCE_N2
    ,ncg.NO_CGCE_N1

    FROM ncm n
    INNER JOIN sh s ON n.CO_SH6 = s.CO_SH6
    INNER JOIN ncm_ppe np ON n.CO_PPE = np.CO_PPE
    INNER JOIN ncm_ppi npi ON n.CO_PPI = npi.CO_PPI
    INNER JOIN ncm_fat_agreg nfa ON n.CO_FAT_AGREG = nfa.CO_FAT_AGREG
    INNER JOIN ncm_cuci nc ON n.CO_CUCI_ITEM = nc.CO_CUCI_ITEM
    INNER JOIN ncm_cgce ncg ON n.CO_CGCE_N3 = ncg.CO_CGCE_N3
    INNER JOIN ncm_isic ni ON n.CO_ISIC_CLASSE = ni.CO_ISIC_CLASSE
),
fat_export_joined AS (
    SELECT 
    fi.VL_FOB
    ,fi.VL_FRETE
    ,fi.VL_SEGURO
    ,fi.KG_LIQUIDO
    ,fi.QT_ESTAT
    ,u.NO_URF
    ,bl.NO_BLOCO
    ,p.CO_PAIS_ISOA3
    ,p.NO_PAIS
    ,v.NO_VIA
    ,fi.CO_NCM
    ,fi.CO_ANO
    ,fi.CO_MES
    ,fi.CO_UNID

FROM bd_becomex.import fi
LEFT JOIN (
    SELECT CO_UNID, MAX(NO_UNID) AS NO_UNID
    FROM ncm_unidade
    GROUP BY CO_UNID
) nu ON fi.CO_UNID = nu.CO_UNID
LEFT JOIN (
    SELECT CO_URF, MAX(NO_URF) AS NO_URF
    FROM urf
    GROUP BY CO_URF
) u ON fi.CO_URF = u.CO_URF
LEFT JOIN (
    SELECT CO_PAIS, MAX(CO_PAIS_ISOA3) AS CO_PAIS_ISOA3, MAX(NO_PAIS) AS NO_PAIS
    FROM paises
    GROUP BY CO_PAIS
) p ON fi.CO_PAIS = p.CO_PAIS
LEFT JOIN (
    SELECT CO_VIA, MAX(NO_VIA) AS NO_VIA
    FROM via
    GROUP BY CO_VIA
) v ON fi.CO_VIA = v.CO_VIA
LEFT JOIN (
    SELECT CO_PAIS, MAX(NO_BLOCO) AS NO_BLOCO
    FROM blocos
    GROUP BY CO_PAIS
) bl ON fi.CO_PAIS = bl.CO_PAIS)

SELECT
    -- Identificação do Produto primeiro (Produto no centro da análise)
    nj.CO_NCM,
    nj.NO_NCM_POR,
    nj.NO_PPE,
    nj.NO_PPI,
    nj.NO_FAT_AGREG,
    
    -- Classificações (ISIC, SH, CUCI, CGCE)
    nj.NO_ISIC_CLASSE,
    nj.NO_ISIC_GRUPO,
    nj.CO_ISIC_DIVISAO,
    nj.NO_ISIC_DIVISAO,
    nj.NO_ISIC_SECAO,

    nj.NO_SH6_POR,
    nj.CO_SH4,
    nj.NO_SH4_POR,
    nj.CO_SH2,
    nj.NO_SH2_POR,
    nj.NO_SEC_POR,
    nj.NO_CUCI_SEC,
    nj.CO_CUCI_GRUPO,
    nj.NO_CUCI_SUB,
    nj.NO_CUCI_ITEM,
    nj.NO_CGCE_N1,
    nj.NO_CGCE_N2,
    nj.NO_CGCE_N3,

    -- Comércio Exterior (valores, peso)
    fi.VL_FOB,
    fi.VL_FRETE,
    fi.VL_SEGURO,
    fi.KG_LIQUIDO,
    fi.QT_ESTAT,
    
    -- Origem/Transporte
    fi.CO_ANO,
    fi.CO_MES,
    fi.CO_UNID,
    fi.NO_PAIS,
    fi.CO_PAIS_ISOA3,
    fi.NO_VIA,
    fi.NO_URF,
    fi.NO_BLOCO
FROM fat_export_joined fi
LEFT JOIN ncm_joined nj ON fi.CO_NCM = nj.CO_NCM


""")


In [0]:
df.write.mode("overwrite").option("mergeSchema", "true").saveAsTable("VW_IMPORTACAO")

In [0]:
df = spark.sql("""

WITH ncm_joined AS (
    SELECT 
    n.CO_NCM
    ,n.NO_NCM_POR
    ,ni.NO_ISIC_CLASSE
    ,ni.NO_ISIC_GRUPO
    ,ni.CO_ISIC_DIVISAO
    ,ni.NO_ISIC_DIVISAO
    ,ni.NO_ISIC_SECAO
    ,s.NO_SH6_POR
    ,s.CO_SH4
    ,s.NO_SH4_POR
    ,s.CO_SH2
    ,s.NO_SH2_POR
    ,s.CO_NCM_SECROM
    ,s.NO_SEC_POR
    ,np.NO_PPE
    ,npi.NO_PPI
    ,nfa.NO_FAT_AGREG
    ,nc.NO_CUCI_ITEM
    ,nc.CO_CUCI_SUB
    ,nc.NO_CUCI_SUB
    ,nc.CO_CUCI_GRUPO
    ,nc.NO_CUCI_SEC
    ,ncg.NO_CGCE_N3
    ,ncg.NO_CGCE_N2
    ,ncg.NO_CGCE_N1
    FROM ncm n
    INNER JOIN sh s ON n.CO_SH6 = s.CO_SH6
    INNER JOIN ncm_ppe np ON n.CO_PPE = np.CO_PPE
    INNER JOIN ncm_ppi npi ON n.CO_PPI = npi.CO_PPI
    INNER JOIN ncm_fat_agreg nfa ON n.CO_FAT_AGREG = nfa.CO_FAT_AGREG
    INNER JOIN ncm_cuci nc ON n.CO_CUCI_ITEM = nc.CO_CUCI_ITEM
    INNER JOIN ncm_cgce ncg ON n.CO_CGCE_N3 = ncg.CO_CGCE_N3
    INNER JOIN ncm_isic ni ON n.CO_ISIC_CLASSE = ni.CO_ISIC_CLASSE
),
fat_export_joined AS (
    SELECT 
    fi.VL_FOB
    ,fi.KG_LIQUIDO
    ,fi.QT_ESTAT
    ,u.NO_URF
    ,bl.NO_BLOCO
    ,p.CO_PAIS_ISOA3
    ,p.NO_PAIS
    ,v.NO_VIA
    ,fi.CO_NCM
    ,fi.CO_ANO
    ,fi.CO_MES
    ,fi.CO_UNID

FROM bd_becomex.export fi
LEFT JOIN (
    SELECT CO_UNID, MAX(NO_UNID) AS NO_UNID
    FROM ncm_unidade
    GROUP BY CO_UNID
) nu ON fi.CO_UNID = nu.CO_UNID
LEFT JOIN (
    SELECT CO_URF, MAX(NO_URF) AS NO_URF
    FROM urf
    GROUP BY CO_URF
) u ON fi.CO_URF = u.CO_URF
LEFT JOIN (
    SELECT CO_PAIS, MAX(CO_PAIS_ISOA3) AS CO_PAIS_ISOA3, MAX(NO_PAIS) AS NO_PAIS
    FROM paises
    GROUP BY CO_PAIS
) p ON fi.CO_PAIS = p.CO_PAIS
LEFT JOIN (
    SELECT CO_VIA, MAX(NO_VIA) AS NO_VIA
    FROM via
    GROUP BY CO_VIA
) v ON fi.CO_VIA = v.CO_VIA
LEFT JOIN (
    SELECT CO_PAIS, MAX(NO_BLOCO) AS NO_BLOCO
    FROM blocos
    GROUP BY CO_PAIS
) bl ON fi.CO_PAIS = bl.CO_PAIS)


SELECT
    -- Produto / Identificação
    nj.CO_NCM,
    nj.NO_NCM_POR,
    nj.NO_PPE,
    nj.NO_PPI,
    nj.NO_FAT_AGREG,
    
    -- Classificações (ISIC, SH, CUCI, CGCE)
    nj.NO_ISIC_SECAO,
    nj.CO_ISIC_DIVISAO,
    nj.NO_ISIC_DIVISAO,
    nj.NO_ISIC_GRUPO,
    nj.NO_ISIC_CLASSE,
    
    nj.NO_SH6_POR,
    nj.CO_SH4,
    nj.NO_SH4_POR,
    nj.CO_SH2,
    nj.NO_SH2_POR,
    nj.NO_SEC_POR,
    
    nj.NO_CUCI_SEC,
    nj.CO_CUCI_GRUPO,
    nj.NO_CUCI_SUB,
    nj.CO_CUCI_SUB,
    nj.NO_CUCI_ITEM,
    
    nj.NO_CGCE_N1,
    nj.NO_CGCE_N2,
    nj.NO_CGCE_N3,
    
    -- Dados comerciais
    fi.VL_FOB,
    fi.KG_LIQUIDO,
    fi.QT_ESTAT,
    
    -- Dados de origem/transporte
    fi.CO_ANO,
    fi.CO_MES,
    fi.CO_UNID,
    fi.NO_PAIS,
    fi.CO_PAIS_ISOA3,
    fi.NO_VIA,
    fi.NO_URF,
    fi.NO_BLOCO

FROM fat_export_joined fi
LEFT JOIN ncm_joined nj ON fi.CO_NCM = nj.CO_NCM

""")


In [0]:
df.count()

In [0]:
df.count()

In [0]:
df.write.mode("overwrite").option("mergeSchema", "true").saveAsTable("VW_EXPORTACAO")

In [0]:
%sql
OPTIMIZE VW_EXPORTACAO;
OPTIMIZE VW_IMPORTACAO;

In [0]:
%sql
create view if not exists vw_exportacao_2024 as
select * from vw_exportacao where co_ano = 2024

In [0]:
%sql
create view if not exists vw_importacao_2024 as
select * from vw_importacao where co_ano = 2024

In [0]:
%sql
select sum(vl_fob) from vw_importacao-- where CO_ANO = 2024

# CORRETO

In [0]:
%sql
select sum(vl_fob) from export where co_ano = 2024

In [0]:
%sql
select sum(vl_fob) from vw_exportacao where co_ano = 2024